In [ ]:
import pyodbc
import time
from IPython.display import clear_output
import winsound as sound
import pandas as pd
with open('login.txt') as key:
    login=key.read().strip()


cnxn = pyodbc.connect(login)
cursor = cnxn.cursor()
    
todays_wo=[]


# main loop which runs as long as the program is running

while 1==1:
    # SQL query for maximo database to download all new workorders. When the Status is 'WAPPR' then it is new 
    # and needs to be coded and assigned to a technician
    sql_query="""
    SELECT wonum, siteid, description 
    FROM workorder
    WHERE  pmnum is null
    and istask = 0
    and (workorder.failurecode <> 'SECURITY' or workorder.failurecode is null)
    and status='WAPPR'
    and convert(date,reportdate) = convert(date,getdate())
    and (siteid = 'BOSTON' or siteid = 'GRAFTON')
    
    ORDER BY reportdate DESC
    ;
    """
    with cursor.execute(sql_query):
        wos=cursor.fetchall()
    
    number_of_wo = len(wos)
    cols=[] # take the df column names from the sql query
    for c in cursor.description:
            cols.append(c[0])
            
    df=pd.DataFrame.from_records(wos, columns=cols) #create df with open work orders
    
    if number_of_wo>0:
        # to get rid of ghost beeps. The query picks up anytime anyone is putting in a work order, even if they are 
        # going to assign it, so the program waits 120 secs to see if they change the status before it beeps.
        clear_output(wait=True)
        print (time.strftime("%H:%M",time.localtime()),'Double checking', end='\r')
        display(df)
        time.sleep(120)
        # after waiting, check the database again.
        with cursor.execute(sql_query):
            wos=cursor.fetchall()
        df=pd.DataFrame.from_records(wos, columns=cols)
        number_of_wo = len(wos)
        
        if number_of_wo==0: #if there are no WOs, then check again
            continue
        else:
            for wo in wos:
                if wo[0] not in todays_wo: #make sure this isn't a work order sitting in WAPPR. If not, beep. Sometimes,
                    # the WO is waiting for more information, so it is kept at WAPPR. So don't beep if this one has been 
                    #seen before and is in the todays_wo list.
                    clear_output(wait=True)
                    print (time.strftime("%H:%M",time.localtime()),number_of_wo, "WOs", end='\r')
                    display(df)
                    sound.Beep(2500,500) #yes, this is an intentionaly annoying beep
                    sound.Beep(2000,1000)
                    time.sleep(60)
                    for w in wos: #add it to the list of WOs today
                        todays_wo.append(w[0])
                    
                    #Check again after a minute and if it's still there
                    with cursor.execute(sql_query):
                        wos=cursor.fetchall()
                    df=pd.DataFrame.from_records(wos, columns=cols)
                    number_of_wo = len(wos)
                    
                    #if new WO is still in the queue, beep again
                    if number_of_wo>0:
                        clear_output(wait=True)
                        print (time.strftime("%H:%M",time.localtime()),number_of_wo, "WOs", end='\r')
                        display(df)
                        sound.Beep(2500,500)
                        sound.Beep(2000,1000)
                        time.sleep(60)
                        
                else: #If this one has been seen before. Ignore it.
                    clear_output(wait=True)
                    print (time.strftime("%H:%M",time.localtime()),number_of_wo, end='\r')
                    display(df)
                    time.sleep(60)
        
    else:
        clear_output(wait=True)
        print (time.strftime("%H:%M",time.localtime()),number_of_wo, end='\r')
        display(df)
        time.sleep(30)



,wonum,siteid,description
0,1635971,GRAFTON,Hi - The weekend of 10-9 to 10-12 i will need ...
1,1635970,GRAFTON,Hi - Weekend of 10-9 to 10-12 i will need a Fa...
2,1635969,GRAFTON,"Hi - on Saturday 09-25, it would be great to h..."
